In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import GlorotNormal
from keras.callbacks import ModelCheckpoint

In [27]:
train_data_path = r"G:\Downloads\attendease_Image_Data_Raw\data_01\train_total\train_total_01"
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(299,299),
        batch_size=128,
        class_mode='categorical',subset='training',shuffle=True)





Found 17150 images belonging to 49 classes.


In [29]:
validate_data_path = r"G:\Downloads\attendease_Image_Data_Raw\data_01\gradient\validation_total_2"
validate_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validate_datagen.flow_from_directory(
    validate_data_path,
    target_size=(299,299),
    batch_size=64,
    class_mode='categorical',shuffle=True) # Use 'validation' subset for validation data

Found 4900 images belonging to 49 classes.


In [31]:

test_data_path = r"G:\Downloads\attendease_Image_Data_Raw\data_01\gradient\test_total"
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(299,299),
        batch_size=32,
        class_mode='categorical',shuffle=True)

Found 4900 images belonging to 49 classes.


In [32]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : 01_Aayushi_Gautam
1 : 02_Abishek_Banerjee
2 : 03_Kaushik_Agarwal
3 : 04_Niyaj_Attar
4 : 05_Avesh_Bhati
5 : 06_Ayan_Datta
6 : 07_Ayush_Mitra
7 : 08_Omkar_Bankar
8 : 09_Prajakta_Bhand
9 : 10_Sahil_Bhosale
10 : 11_Bhuvan_Mangat
11 : 12_tasha_cardozo
12 : 13_Chhavi_Baliyan
13 : 14_Piyush_Dambhare
14 : 15_Khyati_Desai
15 : 16_Priyal_Dighe
16 : 17_Dilpreet_Kaur
17 : 18_Divyanshu_Bisht
18 : 19_Ekta_Chaware
19 : 20_Ashwin_Gadge
20 : 21_Idhika_Vaidya
21 : 22_Aman_Jawalekar
22 : 23_Juned_Manyar
23 : 25_Prachi_Kamble
24 : 26_pranav_katariya
25 : 27_Sourav_Khot
26 : 28_Kinjal_Bandopadhyay
27 : 29_Mallika_Manu
28 : 30_nadar_janice
29 : 31_Santosh_Nagarale
30 : 33_Omkar_Pol
31 : 34_Dhruvi_Panchal
32 : 35_Raj_Aryan
33 : 37_Reetam_Ghosh
34 : 38_Ritwik_Dubey
35 : 39_Raghuram_Sabavath
36 : 40_Sahil_Raj
37 : 41_Sampurna_Mondal
38 : 43_Saranya_Roy
39 : 44_Gaurav_Sharma
40 : 45_Shubham_Shedekar
41 : 46_shivani_shukla
42 : 47_Sh

In [33]:
model_01 = keras.applications.InceptionResNetV2(include_top=True)
print(model_01.summary())

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 149, 149, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                               

In [34]:
model_01.trainable = False

In [35]:
model_01.trainable = False
base_inputs = model_01.layers[0].input
base_outputs = model_01.layers[-1].output

# Modifying the layers with Dense and Dropout layers using GlorotNormal initializer
new_layer_1 = layers.Dense(512, activation="relu", kernel_initializer=GlorotNormal(), name="modified_first_dense_layer")(base_outputs)
new_layer_2 = layers.Dropout(0.3, name="modified_first_dropout_layer")(new_layer_1)
new_layer_3 = layers.Dense(256, activation="relu", kernel_initializer=GlorotNormal(), name="modified_second_dense_layer")(new_layer_2)
new_layer_4 = layers.Dropout(0.2, name="modified_second_dropout_layer")(new_layer_3)
new_layer_5 = layers.Dense(100, activation="relu", kernel_initializer=GlorotNormal(), name="modified_third_dense_layer")(new_layer_4)
classifier = layers.Dense(49, activation="softmax", kernel_initializer=GlorotNormal(), name="Final_output")(new_layer_5)

# Creating the new model
new_model_01 = keras.Model(inputs=base_inputs, outputs=classifier)

print(new_model_01.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 149, 149, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                                           

In [36]:
es_callback = EarlyStopping(monitor='val_loss',patience=2)

In [39]:
new_model_01.compile(optimizer=keras.optimizers.Adam(lr=0.01),loss=keras.losses.categorical_crossentropy,metrics=["accuracy"])

In [24]:
history = new_model_01.fit(train_generator, epochs=50, validation_data=validation_generator,
                       callbacks=[es_callback])

Epoch 1/50
134/134 [==============================] - 11361s 85s/step - loss: 3.1929 - accuracy: 0.1436 - val_loss: 2.3227 - val_accuracy: 0.3476
Epoch 2/50
134/134 [==============================] - 10340s 77s/step - loss: 1.9321 - accuracy: 0.4311 - val_loss: 1.4723 - val_accuracy: 0.5841
Epoch 3/50
134/134 [==============================] - 10360s 77s/step - loss: 1.3811 - accuracy: 0.5857 - val_loss: 1.1065 - val_accuracy: 0.6724
Epoch 4/50
134/134 [==============================] - 10363s 77s/step - loss: 1.0876 - accuracy: 0.6708 - val_loss: 0.8928 - val_accuracy: 0.7251
Epoch 5/50
134/134 [==============================] - 11889s 89s/step - loss: 0.8897 - accuracy: 0.7324 - val_loss: 0.6999 - val_accuracy: 0.8098
Epoch 6/50
134/134 [==============================] - 10369s 77s/step - loss: 0.7380 - accuracy: 0.7785 - val_loss: 0.5852 - val_accuracy: 0.8429
Epoch 7/50
134/134 [==============================] - 11813s 88s/step - loss: 0.6377 - accuracy: 0.8110 - val_loss: 0.5115 -

UnknownError: Graph execution error:

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Downloads\\attendease_Image_Data_Raw\\data_01\\validation_total_2\\46_shivani_shukla\\46_shivani_shukla_115.jpg'
Traceback (most recent call last):

  File "C:\Users\kinja\anaconda3\envs\tf_cpu\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 267, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\kinja\anaconda3\envs\tf_cpu\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\kinja\anaconda3\envs\tf_cpu\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\kinja\anaconda3\envs\tf_cpu\Lib\site-packages\keras\engine\data_adapter.py", line 902, in wrapped_generator

In [ ]:
new_model_01.save('attendance_incptn_resnet.h5')

In [37]:
# Define the checkpoint filepath
checkpoint_filepath = r"G:\Downloads\attendease_Image_Data_Raw\transfer_learning\inception_resnet\attendance_incptn_resnet_{epoch:02d}.h5"

In [38]:
# Define the ModelCheckpoint callback to save the model after every epoch
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch')


In [ ]:
# Fit the model with the checkpoint callback
history = new_model_01.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[es_callback, checkpoint_callback])




Epoch 1/50
134/134 [==============================] - 11712s 87s/step - loss: 3.1785 - accuracy: 0.1586 - val_loss: 2.3020 - val_accuracy: 0.3706
Epoch 2/50
 21/134 [===>..........................] - ETA: 2:02:38 - loss: 2.3010 - accuracy: 0.3426

In [ ]:
# Save the final model
new_model_01.save('attendance_incptn_resnet_final.h5')

In [ ]:
plt.plot(history.history['accuracy'],label="Training accuracy")
plt.plot(history.history['val_accuracy'],label="Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Traing & Validation accuracy curve')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],label="Training Loss")
plt.plot(history.history['val_loss'],label="Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Traing & Validation loss curve')
plt.legend()
plt.show()

In [3]:
model_02 = keras.applications.MobileNetV2(include_top=True)
print(model_02.summary())

14536120/14536120 [==============================] - 3s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
              

In [4]:
model_02.trainable = False

In [5]:
model_02.trainable = False
base_inputs = model_02.layers[0].input
base_outputs = model_02.layers[-1].output

# Modifying the layers with Dense and Dropout layers using GlorotNormal initializer
new_layer_1 = layers.Dense(512, activation="relu", kernel_initializer=GlorotNormal(), name="modified_first_dense_layer")(base_outputs)
new_layer_2 = layers.Dropout(0.3, name="modified_first_dropout_layer")(new_layer_1)
new_layer_3 = layers.Dense(256, activation="relu", kernel_initializer=GlorotNormal(), name="modified_second_dense_layer")(new_layer_2)
new_layer_4 = layers.Dropout(0.2, name="modified_second_dropout_layer")(new_layer_3)
new_layer_5 = layers.Dense(100, activation="relu", kernel_initializer=GlorotNormal(), name="modified_third_dense_layer")(new_layer_4)
classifier = layers.Dense(49, activation="softmax", kernel_initializer=GlorotNormal(), name="Final_output")(new_layer_5)

# Creating the new model
new_model_02 = keras.Model(inputs=base_inputs, outputs=classifier)

print(new_model_02.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [6]:
es_callback = EarlyStopping(monitor='val_loss',patience=2)

In [7]:
new_model_02.compile(optimizer=keras.optimizers.Adam(lr=0.01),loss=keras.losses.categorical_crossentropy,metrics=["accuracy"])

In [13]:
history = new_model_02.fit(train_generator, epochs=50, validation_data=validation_generator,
                       callbacks=[es_callback])

Epoch 1/50
134/134 [==============================] - 1037s 8s/step - loss: 3.5844 - accuracy: 0.0593 - val_loss: 3.2707 - val_accuracy: 0.1020
Epoch 2/50
134/134 [==============================] - 1003s 7s/step - loss: 3.1221 - accuracy: 0.1562 - val_loss: 2.8961 - val_accuracy: 0.2184
Epoch 3/50
134/134 [==============================] - 674s 5s/step - loss: 2.7909 - accuracy: 0.2412 - val_loss: 2.5698 - val_accuracy: 0.3053
Epoch 4/50
134/134 [==============================] - 666s 5s/step - loss: 2.4740 - accuracy: 0.3199 - val_loss: 2.2563 - val_accuracy: 0.3739
Epoch 5/50
134/134 [==============================] - 663s 5s/step - loss: 2.1848 - accuracy: 0.3936 - val_loss: 1.9706 - val_accuracy: 0.4618
Epoch 6/50
134/134 [==============================] - 697s 5s/step - loss: 1.9344 - accuracy: 0.4525 - val_loss: 1.7017 - val_accuracy: 0.5333
Epoch 7/50
134/134 [==============================] - 781s 6s/step - loss: 1.6969 - accuracy: 0.5179 - val_loss: 1.4987 - val_accuracy: 0.57

In [14]:
new_model_02.save('attendance_mobilenet.h5')